# Run AOS on real images

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from flow_analysis_comps.processing.Fourier.OrientationSpaceManager import (
    orientationSpaceManager,
)
import colorcet  # noqa: F401
from flow_analysis_comps.data_structs.AOS_structs import OSFilterParams
from flow_analysis_comps.data_structs.kymographs import (
    graphExtractConfig,
    kymoExtractConfig,
)
from flow_analysis_comps.processing.graph_extraction.graph_extract import (
    VideoGraphExtractor,
)
from flow_analysis_comps.processing.kymographing.kymographer import KymographExtractor
import colorcet as cc

%matplotlib widget

%load_ext autoreload
%autoreload 2



In [ ]:
video_root_folder = Path(
    r"U:\test_data\davis_data\wide_brightfield"
    # r"G:\AMOLF_Data\AMOLF-SHIMIZU Dropbox\Simon van Staalduine\033"
)
graph_data = VideoGraphExtractor(video_root_folder, graphExtractConfig()).edge_data
kymograph_list = KymographExtractor(
    graph_data, kymoExtractConfig()
).processed_kymographs

AOS_params = OSFilterParams(
    space_frequency_center=0.25,
    space_frequency_width=0.3,
    orientation_accuracy=8.0,
    x_spacing=kymograph_list[0].deltas.delta_x,
    y_spacing=kymograph_list[0].deltas.delta_t,
    padding=50
)
AOS_manager = orientationSpaceManager(AOS_params, kymograph_list[0].kymo_no_static)

angles_dict = AOS_manager.get_all_angles()

In [ ]:
angles_response = AOS_manager.refine_all_angles(4.0, angles_dict)

In [ ]:
from flow_analysis_comps.visualizing.kymographs import kymoVisualizer
from flow_analysis_comps.visualizing.AOSFilterVisualizer import AOSVisualizer

AOS_viz = AOSVisualizer(AOS_params)
AOS_viz.demo_image(AOS_manager)

# Test the angular filter

In [ ]:
import numpy as np
from flow_analysis_comps.util.coord_space_util import freqSpaceCoords


coords = freqSpaceCoords(
    (21, 21), x_spacing=1, y_spacing=1
)
theta_show = coords.theta + np.pi
angles = np.arange(5) / 5 * np.pi            
s_a = np.pi / 5

print(angles)

step_1 = theta_show - angles[2]
step_2 = ((step_1 + np.pi) % (2 * np.pi)) - np.pi
posMask = np.abs(step_2) < np.pi / 2
theta_s = step_2 / s_a


angularFilter = 2 * np.exp(-(theta_s**2) / 2)
angularFilter_reversed = np.fft.ifftshift(np.fft.fftshift(angularFilter)[::-1, ::-1])
filterKernel = 0.5 * (angularFilter + angularFilter_reversed)
filterKernel = filterKernel * (1 + 1j * (posMask * 2 - 1))

fig, ax = plt.subplots(1,3)
ax[0].imshow(angularFilter, origin='lower')
ax[1].imshow(angularFilter_reversed, origin='lower')
ax[2].imshow(np.fft.fftshift(filterKernel.real), origin='lower')

# Test with test data

In [ ]:
from flow_analysis_comps.test_data import single_line_img, crossing_line_img
from flow_analysis_comps.util.image_manips import mirror_pad_with_exponential_fade
import numpy as np

single_line_angle = 30

test_single = single_line_img(single_line_angle, std_dev=1, img_size=(101, 101))
test_crossing = crossing_line_img([45, 135], [1, 1])

# test_single = mirror_pad_with_exponential_fade(test_single, pad)
# test_crossing = mirror_pad_with_exponential_fade(test_crossing, pad)

fig, ax = plt.subplots(2)
ax[0].imshow(test_single)
ax[1].imshow(test_crossing)

In [ ]:
AOS_params = OSFilterParams(
    space_frequency_center=.05,
    # space_frequency_width=0.3,
    orientation_accuracy=16.0,
    x_spacing=1,
    y_spacing=1,
    padding=50
)

print(AOS_params.nr_of_samples)
AOS_manager = orientationSpaceManager(AOS_params, test_single)

filter_arrays = AOS_manager.filter_arrays
print(filter_arrays.shape)


angles_dict = AOS_manager.get_max_angles()
nlms_mask = AOS_manager.nlms_simple_case()

In [ ]:
filters = np.moveaxis(filter_arrays, 2, 0)

fig, ax = plt.subplots(len(filters))
for i, filter_ in enumerate(filters):
    ax[i].imshow(np.fft.fftshift(abs(filter_)))

In [ ]:

full_orientation = np.rad2deg(angles_dict)
nlms_orientation = np.where(nlms_mask, np.rad2deg(angles_dict), np.nan)

In [ ]:
from flow_analysis_comps.visualizing.AOSFilterVisualizer import AOSVisualizer
import numpy as np

# angles_max_max = np.nanmax(angles_dict["angles_maxima"].real, axis=0)
# print(angles_dict.keys())

fig, ax = plt.subplot_mosaic([["Orientation", "NLMS_orientation"], ["Histograms", "Histograms"]])
ax["Orientation"].imshow(full_orientation, cmap=cc.cm.rainbow4, vmin=0, vmax=180)
ax["NLMS_orientation"].imshow(nlms_orientation, cmap=cc.cm.rainbow4, vmin=0, vmax=180)
ax["Histograms"].hist(full_orientation.flatten(), bins=180, range=(0, 180), density=True, alpha=.5, label="all pixels", color="red")
ax["Histograms"].hist(nlms_orientation.flatten(), bins=180, range=(0, 180), density=True, alpha=.5, label="NLMS Pixels", color="green")
ax["Histograms"].axvline(45)
ax["Histograms"].axvline(135)
ax["Histograms"].axvline(single_line_angle, c="red")
ax["Histograms"].axvline((single_line_angle+90) % 180, c="red")
ax["Histograms"].legend()


In [ ]:
filter_stack = AOS_manager.response.response_stack

print(filter_stack.shape)

nyquist = int(np.ceil((len(filter_stack) + 1) / 2))
response_frequencies = np.concatenate([np.arange(nyquist), -np.arange(nyquist - 1, 0, -1)])
response_frequencies_full = response_frequencies[..., np.newaxis, np.newaxis]  # Add new axis for broadcasting
response_angles = AOS_manager.response.range_of_angles / np.pi * 180
freq_axis = np.fft.fftshift(response_frequencies)

filter_stack_fft = np.fft.fft(filter_stack, axis=0)
response_fft_derivative1 = filter_stack_fft * (1j * response_frequencies_full)
# response_fft_derivative2 = filter_stack_fft * -(response_frequencies_full**2)

filter_stack_fft_shift = np.fft.fftshift(filter_stack_fft, axes=0)
print(response_frequencies)
print(np.fft.fftshift(response_frequencies))

fig, ax = plt.subplots(len(filter_stack))
for i, img in enumerate(filter_stack):
    ax[i].imshow(img.real, vmin=-.005, vmax=.005)

points = [
    [20, 20], 
    [50, 50], 
    [20, 80], 
    # [50, 0]
    ]

fig, ax = plt.subplots(1, 3, figsize=(10, 5))

ax[0].set_title("Filter Response")
ax[1].set_title("Filter Response FFT")
ax[2].set_title("Derivative Filter Response FFT")

for point in points:
    ax[0].plot(response_angles, filter_stack[:, *point], label=f"{point}")
    ax[1].plot(freq_axis, np.fft.fftshift(filter_stack_fft[:, *point]), label=f"{point}")
    ax[2].plot(freq_axis, np.fft.fftshift(response_fft_derivative1[:, *point]), label=f"{point}")


ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
# print(response_fft_derivative1[:, *points[0]])
sequence_fft = response_fft_derivative1[:, *points[2]]

N = len(sequence_fft)
K = N // 2
c = np.fft.fftshift(sequence_fft)

c /= c[-1]
n = N
companion = np.zeros((n,n), dtype=np.complex128)
companion[1:, :-1] = np.eye(n-1, dtype=np.complex128)
companion[0, 1:] = c[:-1][::-1]
roots = np.linalg.eigvals(companion)
magnitudes = abs(roots)

angles= np.log(roots)
angles = np.rad2deg(np.angle(angles * -1j/2)) 
# angles = np.where(abs(np.log(magnitudes)) < .9, angles, -1)
print(companion)
print(angles)
print(magnitudes)

In [ ]:
angles_all = AOS_manager.get_all_angles()
angles_max = np.nanmax(angles_all["maxima_value"], axis=0)

In [ ]:
plt.figure()
plt.imshow(np.rad2deg(angles_all["angles_maxima"][0]), vmin=0, vmax=360, cmap=cc.cm.rainbow4)